In [2]:
import pandas as pd
from sqlalchemy import create_engine
import configparser
import io
import boto3
import psycopg2

In [3]:
config = configparser.ConfigParser()

# read the configuration file
config.read('multi_config.ini')

# get all the connections
config.sections()

['postgresql', 'aws_s3', 'csv']

In [4]:
'''
Authenticate the Postgres and S3 database by getting the credentials from the config file
'''
database = config.get('postgresql', 'database')
user = config.get('postgresql', 'user')
password = config.get('postgresql', 'password')
host = config.get('postgresql', 'host')
port = config.get('postgresql', 'port')

# AWS Credentials
service_name = config.get('aws_s3', 'service_name')
region_name = config.get('aws_s3', 'region_name')
aws_access_key_id = config.get('aws_s3', 'aws_access_key_id')
aws_secret_access_key = config.get('aws_s3', 'aws_secret_access_key')
s3_bucket = config.get('aws_s3', 's3_bucket')

# check creditials
print("Authentication successful \n")
print(f'The database is "{database}" and the service_name is "{service_name}"')

Authentication successful 

The database is "film_data" and the service_name is "s3"


In [10]:
# load local csv into dataframe
extracted_local_df = pd.read_csv('IMDB-Movie-Data-Local.csv') 
#load csv to dataframe to be loaded into postgres sql
to_load_warehouse_df = pd.read_csv('IMDB-Movie-Data-Postgres.csv') 

In [9]:
## load to postgres
# determine table name
table_name = 'IMDB_movie_data'

# Create an engine instance
alchemyEngine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}', pool_recycle=3600)
# Connect to PostgreSQL server
dbConnection = alchemyEngine.connect()
# Upload data to sql database
to_load_warehouse_df.to_sql(table_name, dbConnection, if_exists='fail')
print(f'PostgreSQL Table, "{table_name}", has been created successfully.')

dbConnection.close()

PostgreSQL Table, "IMDB_movie_data", has been created successfully.


In [8]:
## extract from s3
s3_resource = boto3.resource(
    service_name = service_name,
    region_name = region_name, 
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

string_io = io.BytesIO()
s3_resource.Object(s3_bucket, "IMDB-Movie-Data-S3.csv").download_fileobj(string_io)
s3_contents = string_io.getvalue()

extracted_datalake_df = pd.read_csv(io.BytesIO(s3_contents))
extracted_datalake_df

,Title,Genre,Description,Director
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn
1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott
2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan
3,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet
4,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer
...,...,...,...,...
995,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray
996,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth
997,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu
998,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong


In [11]:
## extract from postgres sql
# Create an engine instance
alchemyEngine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}', pool_recycle=3600);

# Connect to PostgreSQL server
dbConnection = alchemyEngine.connect();

# Read data from PostgreSQL database table and load into a DataFrame instance
sql = f"select * from \"{table_name}\""
extracted_warehouse_df = pd.read_sql(sql, dbConnection);
pd.set_option('display.expand_frame_repr', False);

if dbConnection:
    dbConnection.close()
print("PostgreSQL connection is closed")

PostgreSQL connection is closed
